# いろいろ実験場

# メモリリークの確認
MP4直接読み込み→推論→MP4直接書き込み方式に変更したところ、JupyterNotebookがハングアップする事象が発生。
メモリリークしてswapしているかもしれないので、pythonのmemory profilerを使ってみる。

* https://watlab-blog.com/2022/03/27/memory-profiler/ 

In [4]:
!pip install -U memory_profiler

In [2]:
import cv2
import numpy as np
from memory_profiler import profile

@profile
def mp4_to_mp4():
    print('test')
    try:
        cap = cv2.VideoCapture('images/PXL_20240131_010756099.TS.mp4')

        if cap.isOpened() is False:
            print("can not open camera")
            return
                
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        out = cv2.VideoWriter('tmp/output.mp4', fourcc, fps, (w, h))
        
        while True:
            ret, frame = cap.read()
            if ret:
                out.write(frame)
            else:        
                break
    except Exception as e:
        print(e)
        
    finally:
        cap.release()
        out.release()

mp4_to_mp4()

ERROR: Could not find file /tmp/ipykernel_45146/3635905215.py
test


In [1]:
EDGE_COLOR_TO_RGB = {
    'c' : (0, 255, 255),
    'm' : (228, 0, 127),
    'y' : (255, 255, 0)
}


In [4]:
EDGE_COLOR_TO_RGB.get('c')


(0, 255, 255)

# そもそもMP4を直接読み込んで処理すればよかった節

In [10]:
import cv2
import numpy as np
 
# 0=内蔵カメラ

try:
    cap = cv2.VideoCapture('images/PXL_20240121_053909046.TS.mp4')
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    out = cv2.VideoWriter('tmp/output.mp4', fourcc, fps, (w, h))
    
    while True:
        ret, frame = cap.read()
        if ret:
            out.write(frame)
            print(np.array(frame))
            break        
        else:        
            break
    
finally:
    cap.release()
    out.release()

[[[ 86  97 106]
  [ 86  97 106]
  [ 86  97 106]
  ...
  [ 93 107 113]
  [ 93 107 113]
  [ 93 107 113]]

 [[ 86  97 106]
  [ 86  97 106]
  [ 86  97 106]
  ...
  [ 93 107 113]
  [ 93 107 113]
  [ 93 107 113]]

 [[ 86  97 106]
  [ 86  97 106]
  [ 86  97 106]
  ...
  [ 94 108 114]
  [ 94 108 114]
  [ 94 108 114]]

 ...

 [[ 88 140 180]
  [ 88 140 180]
  [ 88 140 180]
  ...
  [ 81 104 113]
  [ 82 105 114]
  [ 80 103 112]]

 [[ 86 138 178]
  [ 87 139 179]
  [ 87 139 179]
  ...
  [ 81 104 113]
  [ 81 104 113]
  [ 80 103 112]]

 [[ 84 136 176]
  [ 86 138 178]
  [ 87 139 179]
  ...
  [ 81 104 113]
  [ 81 104 113]
  [ 79 102 111]]]


In [2]:
import cv2


# MP4をアニメーションGIFに変換する

https://ito-room.com/python-movie2gif/

In [1]:
import cv2
from PIL import Image

from os.path import dirname,basename

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed


2024-01-28 04:34:17.947859: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-28 04:34:18.052040: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 04:34:18.655582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-28 04:34:18.655747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-28 04:34:18.780206: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def mp4_to_frames(path):
    cap = cv2.VideoCapture(path)
    # fps（ミリ秒）を取得
    fps = cap.get(cv2.CAP_PROP_FPS)

    if not cap.isOpened():
        return None,None

    frames = []

    while True:
        ret, frame = cap.read()
        if ret:
            # BGRをRGBに変換
            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            pillow_image = Image.fromarray(rgb_image)
            frames.append(pillow_image)

        else:
            return frames,fps

In [3]:
def save_gif(frames, save_path, fps):
    # fpsはミリ秒なので、1000をfpsで割り、フレーム間隔を計算
    dur = int(1000.0 / fps)
    # see also: https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
    frames[0].save(save_path, format='GIF', save_all=True, append_images=frames[1:],duration=dur, loop=0 )

In [5]:
mp4_path = 'images/PXL_20240121_053909046.TS.mp4'

save_path = dirname(mp4_path) + '/' + basename(mp4_path).split('.')[0]+'.gif'

frames, fps = mp4_to_frames(mp4_path)


In [106]:
def mp4_to_frames2(path, max_frame_per_sec=10, do_resize=True, max_long_side=1000):
    cap = cv2.VideoCapture(path)
    
    if not cap.isOpened():
        return None,None
    
    # fpsを取得
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 何枚おきにフレームを取得するか計算
    per_frame = math.ceil(fps / max_frame_per_sec)
    print(f'per_frame:{per_frame}')
    
    # フレームの間隔（ミリ秒）を計算
    dur = int(1000 / fps * per_frame)
    print(f'dur:{dur}')
    
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if ret:
            # フレームの間引き
            if (i % per_frame) != 0:
                i += 1
                continue
            
            # BGRをRGBに変換
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pillow_image = Image.fromarray(rgb_frame)
            
            if do_resize:
                w, h = cal_resided_w_h(pillow_image.width,pillow_image.height,max_long_side)
                pillow_image = pillow_image.resize((w, h))
            frames.append(pillow_image)
            i += 1
        else:
            return frames,dur


def cal_resided_w_h(w,h,max_long_side):
    if w > max_long_side or h > max_long_side:
        if w > h:
            ret_w = max_long_side
            ret_h = int(h * max_long_side / w)
        else:
            ret_h = max_long_side
            ret_w = int(w * max_long_side / h)
    else:
        ret_w, ret_h = w,h
    return ret_w, ret_h


    

In [98]:
def save_gif2(frames, save_path, dur):
    # see also: https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
    frames[0].save(save_path, format='GIF', save_all=True, append_images=frames[1:],duration=dur, loop=0 )

In [107]:
%%time
# mp4->PillowImageのリストへの変換
mp4_path = 'images/PXL_20240121_053909046.TS.mp4'
save_path = dirname(mp4path) + '/' + basename(mp4path).split('.')[0]+'.gif'
frames, dur = mp4_to_frames2(mp4_path)


per_frame:3
dur:100
CPU times: user 11.7 s, sys: 408 ms, total: 12.1 s
Wall time: 4.06 s


In [108]:
%%time
# メモリ上のデータをGIFファイルに変換
save_gif2(frames, save_path, dur)

CPU times: user 19.2 s, sys: 0 ns, total: 19.2 s
Wall time: 19.3 s


# MP4に保存 moviepy

In [140]:
def mp4_to_frames3(path, max_frame_per_sec=10, do_resize=True, max_long_side=1000):
    cap = cv2.VideoCapture(path)
    
    if not cap.isOpened():
        return None,None
    
    # fpsを取得
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 何枚おきにフレームを取得するか計算
    per_frame = math.ceil(fps / max_frame_per_sec)
    print(f'per_frame:{per_frame}')
    
    # フレームの間隔（ミリ秒）を計算
    dur = int(1000 / fps * per_frame)
    print(f'dur:{dur}')
    
    frames = []
    i = 0
    while True:
        ret, frame = cap.read()
        if ret:
            # フレームの間引き
            if (i % per_frame) != 0:
                i += 1
                continue
            
            # BGRをRGBに変換
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            i += 1
        else:
            return frames,dur
    

In [127]:
frames, dur = mp4_to_frames2(mp4_path)

per_frame:3
dur:100


65

# moviepyでVideFileClipを使えば、GIF->MP4変換は可能

In [4]:
import moviepy.editor as mp

imput_gif='images/PXL_20240121_053909046.gif'
output_mp4='images/PXL_20240121_053909046_gif.mp4'

with mp.VideoFileClip(imput_gif) as movie_file:
    movie_file.write_videofile(output_mp4)
    movie_file.close()




Moviepy - Building video images/PXL_20240121_053909046_gif.mp4.
Moviepy - Writing video images/PXL_20240121_053909046_gif.mp4



Moviepy - Done !
Moviepy - video ready images/PXL_20240121_053909046_gif.mp4


# moviepyでframesからMP4ファイルを直接出力することはできなかった。

* https://github.com/Zulko/moviepy/issues/1986
* https://github.com/Zulko/moviepy/issues/1990

2024/1/28時点でリリース1.03には修正内容は含まれていない。パッチを当てたいが、どこが修正されているのか追えなかった。


In [3]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

def make_frame(t):
    # 座標(t,t)に長方形を描画
    fig, ax = plt.subplots(figsize=(6,6))
    ax.add_patch(Rectangle((t, t), 1,2))
    ax.set_xlim(0,10)
    ax.set_ylim(0,10)
    # bitmapに変換. bmp.shape==(432,432,3)
    bmp = mplfig_to_npimage(fig)
    plt.close()
    return bmp

fname = "tmp/out2.mp4"
# 10秒, 30fps の計300フレームからなる動画を生成
with VideoClip(make_frame, duration = 10) as animation:
    animation.fps = animation.set_fps(30)
    animation.write_videofile(fname)


Moviepy - Building video tmp/out2.mp4.
Moviepy - Writing video tmp/out2.mp4



TypeError: must be real number, not VideoClip

# ImageIOでも失敗

In [133]:
import imageio
import numpy as np

In [120]:
mp4out = 'tmp/animation.mp4'

In [131]:
frame = frames[0]
type(frame)

PIL.Image.Image

In [139]:
imageio.mimwrite('tmp/imageio.gif', frames)

In [138]:
imageio.help()

TIFF - TIFF format [.tif .tiff .stk .lsm]
BMP-PIL - Windows Bitmap via Pillow [.bmp]
BUFR-PIL - BUFR via Pillow [.bufr]
CUR-PIL - Windows Cursor via Pillow [.cur]
DCX-PIL - Intel DCX via Pillow [.dcx]
DDS-PIL - DirectDraw Surface via Pillow [.dds]
DIB-PIL - Windows Bitmap via Pillow []
EPS-PIL - Encapsulated Postscript via Pillow [.ps .eps]
FITS-PIL - FITS via Pillow [.fit .fits]
FLI-PIL - Autodesk FLI/FLC Animation via Pillow [.fli .flc]
FPX-PIL - FlashPix via Pillow [.fpx]
FTEX-PIL - Texture File Format (IW2:EOC) via Pillow [.ftc .ftu]
GBR-PIL - GIMP brush file via Pillow [.gbr]
GIF-PIL - Compuserve GIF via Pillow [.gif]
GRIB-PIL - GRIB via Pillow [.grib]
HDF5-PIL - HDF5 via Pillow [.h5 .hdf]
ICNS-PIL - Mac OS icns resource via Pillow [.icns]
ICO-PIL - Windows Icon via Pillow [.ico]
IM-PIL - IFUNC Image Memory via Pillow [.im]
IMT-PIL - IM Tools via Pillow []
IPTC-PIL - IPTC/NAA via Pillow [.iim]
JPEG-PIL - JPEG (ISO 10918) via Pillow [.jfif .jpe .jpg .jpeg]
JPEG2000-PIL - JPEG 2000 

In [134]:
video_writer = imageio.get_writer(mp4out, fps=10)
try:
    for frame in frames:
        video_writer.append_data(np.array(frame))
finally:
    video_writer.close()
    

TypeError: TiffWriter.write() got an unexpected keyword argument 'fps'

# 以下、雑多難実験の場所

In [113]:
image_path = 'images/dance_input.gif'
#image_path ='images/pexels-photo-4384679.jpeg'
images = tf.io.read_file(image_path)
images = tf.image.decode_gif(images)


In [116]:
Image.open(image_path).info['duration']

100

In [119]:
import math


interval = math.ceil(15/10)
dur = 1000/fps * interval
0 % 2
i = 0
i +=3

i +=3
i

6